In [ ]:
!pip install neptune

In [ ]:
!pip install rouge

In [ ]:
! pip install datasets

In [ ]:
import neptune

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import os

In [ ]:
from datasets import load_dataset


dataset = load_dataset('huggingartists/placebo')

In [ ]:
dataset['train'][2]['text']

'If I only could, Id be running up that hill\nIf I only could, Id be running up that hill\nIt doesnt hurt me\nYou want to feel how it feels?\nYou want to know, know that it doesnt hurt me?\nYou want to hear about the deal Im making?\nYou \nYou and me \nAnd if I only could\nMake a deal with God\nId get him to swap our places\nBe running up that road\nBe running up that hill\nBe running up that building\nIf I only could, oh\nYou dont want to hurt me\nBut see how deep the bullet lies\nUnaware that Im tearing you asunder\nTheres a thunder in our hearts, baby\nSo much hate for the ones we love\nTell me, we both matter, dont we?\nYou \nYou and me \nYou and me wont be unhappy\nAnd if I only could\nMake a deal with God\nId get him to swap our places\nBe running up that road\nBe running up that hill\nBe running up that building\nIf I only could, oh\nCmon, baby, cmon, cmon darling\nLet me steal this moment from you now\nCmon, angel, cmon, cmon, darling\nLets exchange the experience, oh\nAnd if I

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df.head(1)

,text
0,"Sucker love is heaven sent\nYou pucker up, our..."


In [ ]:
import re

def preprocess_text(text, replacement=' '):
    text = text.replace('\u2005', ' ')
    text = text.replace('\n\n', '. ')
    text = re.sub(r'[\u2000-\u206F]', replacement, text)
    text = text.replace('&quot;', ' ')
    return text

In [ ]:
df['text'] = df['text'].apply(preprocess_text)

In [ ]:
df.head(1)

,text
0,"Sucker love is heaven sent\nYou pucker up, our..."


In [ ]:
len(df)

255

In [ ]:
from sklearn.model_selection import train_test_split


train, test = train_test_split(df, test_size=0.1, random_state=42)


In [ ]:
def prepare_data(df, num_last_words=20):

    df['prompt'] = df['text'].apply(lambda x: ' '.join(x.replace('\n', '. ').split()[:-num_last_words]))
    df['target'] = df['text'].apply(lambda x: ' '.join(x.replace('\n', '. ').split()[-num_last_words:]))
    return df[['prompt', 'target']]


In [ ]:
test_pt = prepare_data(test)

In [ ]:
train['text'] = train['text'].str.replace('\n', '. ', regex=False)

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
from transformers import GPT2Tokenizer
import torch
from torch.utils.data import Dataset

class SongLyrics(Dataset):
    def __init__(self, dataframe, control_code, gpt2_type="gpt2"):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.lyrics = []


        for row in dataframe['text']:

            encoded_text = self.tokenizer.encode(f"<|{control_code}|>{row} <|endoftext|>",
                                                 truncation=True, max_length=1024)
            self.lyrics.append(torch.tensor(encoded_text))

        self.lyrics_count = len(self.lyrics)

    def __len__(self):
        return self.lyrics_count

    def __getitem__(self, item):
        return self.lyrics[item]


In [ ]:
dataset = SongLyrics(train, control_code="genre:punk")

In [ ]:
first_item = dataset[0]
print(first_item)


print(tokenizer.decode(first_item.numpy()))

tensor([   27,    91, 35850,    25, 30354,    91,    29, 21478,   307,   534,
         1660, 39153,   345,  3424,    13,   383,  8122,  3704,    13,  5821,
          307,   534, 28475,   345,   297, 18044,   502,   287,    13,   921,
        28329,  2650,    13,  3894,   314,   303,  1775,   345,  8659,    11,
          314,   303,  1775,   345,  3960,    13,   383,  2187,  1755,   832,
           13,  1406,  5821,   307,   534,  1660, 39153,   345,  3424,    13,
        21020,  4171,    13,  5821,   307,   534,  2988,    11,  5821,   307,
          534,  2802,    13,  5821,   307,   534, 18854,    11,  5821,   307,
        12431,    13,  5821,   307,   534,  2988,    11,  5821,   307,   534,
         2802,    13,  5821,   307,   534, 18854,    11,  5821,   307, 12431,
           13,  5821,   307,   534, 20030, 39153,   534,  5848,    13,  2080,
        13135, 29294,  5899,    13,   843,  5821,   275, 16843,   534, 18021,
          345,   297,  1239,   443,   615, 16843,    13,   911, 

In [ ]:
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

def train_model(
    dataset, model, tokenizer, run,
    batch_size=16, epochs=20, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False, save_model_on_epoch=False,
):
    device = torch.device("cuda")
    model = model.cuda()
    model.train()
    acc_steps = 100

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss = 0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):
        epoch_loss = 0
        print(f"Training epoch {epoch+1}")
        progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch+1}/{epochs}")

        for idx, entry in progress_bar:
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
            epoch_loss += loss.item()
            progress_bar.set_postfix({'loss': loss.item()})

        avg_loss = epoch_loss / len(train_dataloader)
        run[f'training/avg_loss_epoch_{epoch+1}'].log(avg_loss)
        print(f'Epoch {epoch+1} Average Loss: {avg_loss:.4f}')

        if save_model_on_epoch:
            model_save_path = f"{output_dir}/{output_prefix}-epoch{epoch+1}.pt"
            torch.save(model.state_dict(), model_save_path)
            run[f"model_checkpoints/epoch_{epoch+1}"].upload(model_save_path)

    final_model_save_path = f"{output_dir}/{output_prefix}-final.pt"
    torch.save(model.state_dict(), final_model_save_path)
    run["model/final"].upload(final_model_save_path)

    return model


In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=15,
    entry_length=20,
    top_p=0.7,
    temperature=0.7,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break

            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>"
              generated_list.append(output_text)

    return generated_list

In [ ]:
def text_generation(test_data):
    generated_lyrics = []
    for _, row in test_data.iterrows():
        prompt_text = row['prompt']
        generated_text = generate(model.to('cpu'), tokenizer, prompt_text, entry_count=1)
        generated_lyrics.append(generated_text)
    return generated_lyrics

In [ ]:
filtered_test_pt = test_pt[(test_pt['prompt'].str.strip() != '') & (test_pt['target'].str.strip() != '')]

In [ ]:
def remove_prompt(row):
    prompt = row['prompt']
    generated = row['generated']
    if generated.startswith(prompt):
        return generated[len(prompt):]
    return generated

In [ ]:
def clean_generated(text):
    text = re.sub(r'^[^\w]+', '', text)
    last_dot_index = text.rfind('.')
    if last_dot_index != -1:
        text = text[:last_dot_index + 1]

    return text

In [ ]:
def combine_df(test_df, generated_lyrics):
  lyrics = [' '.join(lyric) for lyric in generated_lyrics]
  new_df = pd.DataFrame({
        'target': test_df['target'],
        'prompt': test_df['prompt'],
        'generated': lyrics
    })
  new_df['generated'] = new_df.apply(remove_prompt, axis = 1)
  new_df['generated'] = new_df['generated'].apply(clean_generated)


  return new_df



In [ ]:
import statistics
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from rouge import Rouge

In [ ]:
def compute_metrics_log(df, run):
    bleu_scores = []

    rouge = Rouge()

    for i in range(len(df)):
        reference = [word_tokenize(df['target'].iloc[i])]
        candidate = word_tokenize(df['generated'].iloc[i])  #
        bleu = sentence_bleu(reference, candidate, smoothing_function=SmoothingFunction().method1)
        bleu_scores.append(bleu)



    avg_bleu_score = np.mean(bleu_scores)
    run["evaluation/avg_bleu"] = avg_bleu_score
    rouge_scores = rouge.get_scores(df['generated'].tolist(), df['target'].tolist(), avg=True)


    for key, values in rouge_scores.items():
        run[f"evaluation/avg_{key}_f"] = values['f']
        run[f"evaluation/avg_{key}_p"] = values['p']
        run[f"evaluation/avg_{key}_r"] = values['r']

    print(f"Logged BLEU: {avg_bleu_score}")
    for key, values in rouge_scores.items():
      print(f"Logged {key}: {values}")






In [ ]:
hyperparameters = [
    {'batch_size': 16, 'lr': 3e-5, 'epochs': 20},
    {'batch_size': 32, 'lr': 2e-3, 'epochs': 15},
    {'batch_size': 16, 'lr': 2e-3, 'epochs': 10},
    {'batch_size': 16, 'lr': 2e-4, 'epochs': 20}

]



In [ ]:
project = "litichewski/lsml-final"
token = "eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmOTA2YTU3MS03M2Q4LTQ2NWItOGRjMS1kY2YxMTU1MmMzMDkifQ=="

In [ ]:
model_reg = neptune.init_model(
    name="Lyrics generator",
    key="MOD",
    project=project,
    api_token=token
)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/litichewski/lsml-final/m/LSMLFIN-MOD


In [ ]:
best_model = None
max_bleu = 0

for params in hyperparameters:
    with neptune.init_run(
        project = project,
        api_token = token,
    ) as run:
      run["parameters"] = params

      model = GPT2LMHeadModel.from_pretrained("gpt2")
      tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
      model = train_model(dataset, model, tokenizer, run, batch_size=params['batch_size'], epochs=params['epochs'], lr=params['lr'])
      generated_lyrics = text_generation(filtered_test_pt)
      df  =  combine_df(filtered_test_pt, generated_lyrics)

      compute_metrics_log(df, run)

      run.wait()

      current_bleu = run["evaluation/avg_bleu"].fetch()


      model_name = f"gpt-placebo-{run['sys/id'].fetch()}.pt"
      torch.save(model.state_dict(), model_name)


      version = neptune.init_model_version(model = "LSMLFIN-MOD", project = project, api_token = token)

      version['model'].upload(model_name)

      version['hyperparameters'] = params

      version['eval/bleu'] = current_bleu



      if current_bleu > max_bleu:
        if best_model is not None:
          best_model.change_stage('staging')
        version.change_stage('production')
        max_bleu = current_bleu
        best_model = version
      else:
        version.change_stage('staging')







[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/litichewski/lsml-final/e/LSMLFIN-2
Training epoch 1


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20:   0%|          | 0/229 [00:00<?, ?it/s]

Epoch 1 Average Loss: 0.8591
Training epoch 2


Epoch 2/20:   0%|          | 0/229 [00:00<?, ?it/s]

...

Epoch 19 Average Loss: 0.5002
Training epoch 20


Epoch 20/20:   0%|          | 0/229 [00:00<?, ?it/s]

Epoch 20 Average Loss: 0.5266


100%|██████████| 1/1 [00:31<00:00, 31.09s/it]


Logged BLEU: 0.3328947551564273
Logged rouge-1: {'r': 0.6893964411921069, 'p': 0.685748597394167, 'f': 0.6212938775868663}
Logged rouge-2: {'r': 0.5840471468675803, 'p': 0.6313763144948035, 'f': 0.5492470054532964}
Logged rouge-l: {'r': 0.6893964411921069, 'p': 0.685748597394167, 'f': 0.6212938775868663}
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/litichewski/lsml-final/m/LSMLFIN-MOD/v/LSMLFIN-MOD-4
[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/litichewski/lsml-final/e/LSMLFIN-5/metadata


In [ ]:
model_reg.stop()

[neptune] [info   ] Shutting down background jobs, please wait a moment...
[neptune] [info   ] Done!
[neptune] [info   ] All 0 operations synced, thanks for waiting!
[neptune] [info   ] Explore the metadata in the Neptune app: https://app.neptune.ai/litichewski/lsml-final/m/LSMLFIN-MOD/metadata
